In [1]:
import tensorflow as tf

In [2]:
def weight_variable(shape, name=None):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name)
 
def bias_variable(shape, name=None):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name)
 
def conv2d(input, filter, strides, padding="SAME", name=None):
    # filters with shape [filter_height * filter_width * in_channels, output_channels]
    # Must have strides[0] = strides[3] =1
    # For the most common case of the same horizontal and vertices strides, strides = [1, stride, stride, 1]
    '''
    Args:
        input: A Tensor. Must be one of the following types: float32, float64.
        filter: A Tensor. Must have the same type as input.
        strides: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input.
        padding: A string from: "SAME", "VALID". The type of padding algorithm to use.
        use_cudnn_on_gpu: An optional bool. Defaults to True.
        name: A name for the operation (optional).
    '''
    return tf.nn.conv2d(input, filter, strides, padding=padding, name=name)  # padding="SAME"用零填充边界

def Conv(input, name, filter_size, bias_size, stride, padding):
    with tf.name_scope(name):
        with tf.name_scope('Variable'):
            filters = weight_variable(filter_size, name='filter')
            bias = weight_variable(bias_size, name='bias')
        with tf.name_scope("Convolution"):
            layer = tf.nn.relu(conv2d(input, filters, strides=stride, padding = padding) + bias)
    return layer

In [3]:
X = tf.placeholder(tf.float32, [None, 299, 299, 3], name="Input")

In [10]:
with tf.name_scope('steam'):
        net = Conv(input=X, name = 'Layer1', filter_size = [3,3,3,32], bias_size = [32],padding = 'VALID', stride = [1,2,2,1])
        net = Conv(input=net, name = 'Layer2', filter_size = [3,3,32,32], bias_size = [32], padding = 'VALID', stride = [1,1,1,1])
        net = Conv(input=net, name = 'Layer3', filter_size = [3,3,32,64], bias_size = [64], padding = 'SAME', stride = [1,1,1,1])
        with tf.name_scope('MaxPool_1'):
            maxpool = tf.nn.max_pool(net, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID",name='maxpool')
        net = Conv(input=net, name = 'Layer4', filter_size = [3,3,64,96], bias_size = [96], padding="VALID", stride = [1,2,2,1])
        with tf.name_scope('Filter_concat_1'):
            net = tf.concat([maxpool, net], 3)

In [11]:
net

<tf.Tensor 'steam_2/Filter_concat_1/concat:0' shape=(?, 73, 73, 160) dtype=float32>

In [13]:
net_1 = Conv(input=net, name = 'Layer5_1', filter_size = [1,1,160,64], bias_size = [64], stride = [1,1,1,1])
net_1 = Conv(input=net_1, name = 'Layer6_1', filter_size = [3,3,64,96], bias_size = [96], padding="VALID",stride = [1,1,1,1])
net_2 = Conv(input=net, name = 'Layer5_2', filter_size = [1,1,160,64], bias_size = [64], stride = [1,1,1,1])
net_2 = Conv(input=net_2, name = 'Layer6_2', filter_size = [7,1,64,64], bias_size = [64], stride = [1,1,1,1])
net_2 = Conv(input=net_2, name = 'Layer7_2', filter_size = [1,7,64,64], bias_size = [64], stride = [1,1,1,1])
net_2 = Conv(input=net_2, name = 'Layer8_2', filter_size = [3,3,64,96], bias_size = [96], padding="VALID", stride = [1,1,1,1])

TypeError: Conv() missing 1 required positional argument: 'padding'